# Imports

In [1]:
import numpy as np
import pandas as pd

from utilities import to_float, get_path

In [2]:
DATA_PATH = get_path()

In [3]:
df_tweets = pd.read_csv(DATA_PATH + 'tweets.csv')

In [4]:
df_users = pd.read_csv(DATA_PATH + 'users.csv')

In [5]:
df_users.head()

,id,name,lang,bot,created_at,statuses_count
0,2353593986,Lamonica Raborn,en,1,2019-02-22 18:00:42,76.0
1,2358850842,Lourie Botton,en,0,2019-02-26 03:02:32,54.0
2,137959629,Dadan Syarifudin,en,1,2015-04-30 07:09:56,3.0
3,466124818,Carletto Focia,it,1,2017-01-18 02:49:18,50.0
4,2571493866,MBK Ebook,en,0,2019-06-18 19:30:21,7085.0


In [6]:
df_users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11508 entries, 0 to 11507
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   id              11508 non-null  int64  
 1   name            11507 non-null  object 
 2   lang            11508 non-null  object 
 3   bot             11508 non-null  int64  
 4   created_at      11508 non-null  object 
 5   statuses_count  11109 non-null  float64
dtypes: float64(1), int64(2), object(3)
memory usage: 539.6+ KB


Check if any user record has a null value in it.

Extract the only record with a null name (id = 1535).

In [7]:
df_null = df_users.isnull()
idx_null = df_users.index[df_null["name"] == True].tolist()
df_users.iloc[idx_null]

,id,name,lang,bot,created_at,statuses_count
1535,2166124159,NaN,en,0,2018-11-02 06:39:14,6566.0


In [8]:
df_tweets.info(null_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13664696 entries, 0 to 13664695
Data columns (total 10 columns):
 #   Column          Non-Null Count     Dtype 
---  ------          --------------     ----- 
 0   id              13664694 non-null  object
 1   user_id         13447413 non-null  object
 2   retweet_count   13227562 non-null  object
 3   reply_count     13016818 non-null  object
 4   favorite_count  13017154 non-null  object
 5   num_hashtags    12607172 non-null  object
 6   num_urls        13016073 non-null  object
 7   num_mentions    12810531 non-null  object
 8   created_at      13664696 non-null  object
 9   text            13126975 non-null  object
dtypes: object(10)
memory usage: 1.0+ GB


C:\Users\al\AppData\Local\Temp\ipykernel_10368\3419106329.py:1: FutureWarning: null_counts is deprecated. Use show_counts instead
  df_tweets.info(null_counts=True)


In [9]:
# Total number of tweets, including possible duplicates
df_tweets.size

136646960

## Check for duplicate records

Almost 2 million tweets are exact duplicates.

In [10]:
df_tweets.duplicated().value_counts()

False    11712597
True      1952099
dtype: int64

#  Null user_id and text
We remove the records that have invalid user_id and text, since these records can't be used neither to analyze user behaviour, neither to perform any kind of topic analysis on the text. 

The only utility that they can have could be related to the twitter density for a given period of time, but we go for the deletion since 56280 is not a great amount of records w.r.t. the total number of tweets. Also, notice that the counters have several null values.

In [11]:
df_tweets = df_tweets.drop(df_tweets.index[df_tweets.user_id.isnull() & df_tweets.text.isnull()])

## File Checkpoint

In [ ]:
df_tweets.to_csv(path_or_buf=DATA_PATH + 'tweets_no_dupl.csv', sep='#', index=False)

In [ ]:
df_tweets = pd.read_csv(DATA_PATH + 'tweets_no_dupl.csv', sep='#')

## 1.1 Drop duplicates over all columns
First, we drop the duplicates by all attributes

In [ ]:
df_tweets = df_tweets.drop_duplicates(keep='first')
df_tweets.duplicated().value_counts()

In [ ]:
df_tweets.info(show_counts=True)

## File checkpoint

In [ ]:
df_tweets.to_csv(path_or_buf=DATA_PATH + 'tweets_no_dupl_1.1.csv', sep='#', index=False)

In [ ]:
df_tweets = pd.read_csv(DATA_PATH + 'tweets_no_dupl_1.1.csv', sep='#')

# Checking for duplicates on a subset of columns

### ID
The are a considerable amount of tweets with same id, but we can't consider them duplicates since the id has no semantic that we can check.

In [ ]:
df_tweets.duplicated(subset=['id']).value_counts()

In [ ]:
df_tweets.duplicated(subset=['id', 'user_id']).value_counts()

In [ ]:
df_tweets.duplicated(subset=['id', 'user_id', 'text']).value_counts()

In [ ]:
df_tweets.duplicated(subset=['id', 'user_id', 'created_at', 'text']).value_counts()

# 1.2 ['user_id', 'text','created_at']
Considering the triple  `['user_id', 'text','created_at']`, we can see that about 10% of the data consists in duplicated records and we are not able to detect them if we include the `id` value in the duplicate definition.

In [ ]:
# Criterion for duplicates
duplicates_bool = df_tweets.duplicated(subset=['user_id', 'text', 'created_at'], keep=False)
df_tweets[duplicates_bool].info(show_counts=True)

We can observe that the other columns of these duplicates have a considerable amount of null values. 

In order to keep the correct values, we proceed by performing a merge of the copies, keeping the non null value (usually the max value).

We first convert the counts and nums to numeric type, setting to -1 the meaningless values.
Then, the merge will proceed by taking the max over the duplicates.

In [ ]:
for attr in ['retweet_count', 'reply_count', 'favorite_count', 'num_hashtags', 'num_urls', 'num_mentions', 'user_id']:
    df_tweets[attr + '_conv'] = df_tweets[attr].apply(to_float)

# recreate the index column for the groupby
df_tweets = df_tweets.reset_index()
df_tweets.index = df_tweets['index']

In [ ]:
aggregated = df_tweets[duplicates_bool].groupby(['user_id', 'text', 'created_at']).agg(
    {'retweet_count_conv': max, 'reply_count_conv': max, 'favorite_count_conv': max,
     'num_hashtags_conv': max, 'num_urls_conv': max, 'num_mentions_conv': max,
     'id': tuple, 'index': min}).reset_index()

In [ ]:
aggregated.index = aggregated['index']

In [ ]:
attributes = ['retweet_count_conv', 'reply_count_conv', 'favorite_count_conv', 'num_hashtags_conv', 'num_mentions_conv',
              'num_urls_conv']

# Saves index and the attributes
df_tweets.loc[aggregated.index, attributes] = aggregated[attributes]

In [ ]:
# reset the -1 to NaN to check how many null values have been removed
for a in attributes:
    df_tweets[a] = df_tweets[a].replace(-1, np.NaN)

df_tweets[duplicates_bool].info(show_counts=True)

In [ ]:
df_tweets.drop_duplicates(['user_id', 'text', 'created_at'], inplace=True)

In [ ]:
# check that the non nan copies have been kept
df_tweets[df_tweets.index.isin(aggregated.index)].info(show_counts=True)

Everything seems fine, so we rename the 'conv' columns to the original name and drop the uncleaned ones.

In [ ]:
dropped = df_tweets.drop(
    columns=['retweet_count', 'reply_count', 'favorite_count', 'num_hashtags', 'num_urls', 'num_mentions', 'index'])

In [ ]:
df_tweets = dropped.rename(columns={'retweet_count_conv': 'retweet_count',
                                    'reply_count_conv': 'reply_count',
                                    'favorite_count_conv': 'favorite_count',
                                    'num_hashtags_conv': 'num_hashtags',
                                    'num_urls_conv': 'num_urls',
                                    'num_mentions_conv': 'num_mentions'})

In [ ]:
df_tweets = df_tweets.reset_index()
df_tweets = df_tweets.drop(columns=['index'])

In [ ]:
df_tweets.info(show_counts=True)

## File checkpoint

In [ ]:
df_tweets.to_csv(path_or_buf=DATA_PATH + 'tweets_no_dupl_user_text_createdat.csv', sep='#', index=False)

In [ ]:
df_tweets = pd.read_csv(DATA_PATH + 'tweets_no_dupl_user_text_createdat.csv', sep='#')

# Check how many duplicates we have for each attribute

In [ ]:
df_tweets.info(show_counts=True)

There are no records where both the user_id and text fields are null

In [ ]:
df_tweets[df_tweets.user_id.isnull() & df_tweets.text.isnull()].info()

# 1.3 [text, created_at]
We now consider the situation where `text` and `created_at` are equal among different records. 

Despite it is certainly possible that two users post the same text at the same time, the situation is suspicious, and we check the possibility that the two records are the same tweet from the same user, but with mispelled `user_id`.

For this analysis we consider only the tweets with non-null text.

In [ ]:
# Criterion for duplication
duplicates_bool = ~df_tweets.text.isnull() & df_tweets.duplicated(subset=['text', 'created_at'], keep=False)
duplicates_bool.value_counts()

In [ ]:
df_tweets[duplicates_bool & (df_tweets.user_id_conv == -1)].info(show_counts=True)

We consider a NaN or alphanumeric `user_id` to be wrong, since the correct format is the one with only numerical characters.
Out of the 646625 duplicates by `[text,created_at]`, 321006 have wrong `user_id`.
We proceed by keeping the copy with the right `user_id`, and selecting the counter attributes by a max operation, as we did at step **1.2**.

In [ ]:
for a in ['retweet_count', 'reply_count', 'favorite_count', 'num_hashtags', 'num_urls', 'num_mentions']:
    df_tweets[a] = df_tweets[a].replace(np.NaN, -1)

In [ ]:
# recreate the index column for the groupby
df_tweets = df_tweets.reset_index()
df_tweets.index = df_tweets['index']

In [ ]:
def select_best_user_id(user_ids):
    greater_than_zero = []
    smaller_than_zero = []
    for id in user_ids:
        if id > 0:
            greater_than_zero.append(id)
        else:
            smaller_than_zero.append(id)

    # if all but one are < 0, keep the one that is > 0
    if len(greater_than_zero) == 1:
        return greater_than_zero[0]
    if len(greater_than_zero) > 1:
        return tuple(greater_than_zero)

In [ ]:
aggregated = df_tweets[duplicates_bool].groupby(['text', 'created_at']).agg(
    {'retweet_count': max, 'reply_count': max, 'favorite_count': max,
     'num_hashtags': max, 'num_urls': max, 'num_mentions': max,
     'user_id_conv': select_best_user_id,
     'index': min}).reset_index()

In [ ]:
aggregated.index = aggregated['index']

In [ ]:
# Count how many duplicates were from the same user but with misspelled user_id (== True)
tweets_with_valid_user_id = aggregated.user_id_conv.apply(type) == float
tweets_with_valid_user_id.value_counts()

In [ ]:
aggregated_one_text = aggregated[tweets_with_valid_user_id]

In [ ]:
attributes = ['retweet_count', 'reply_count', 'favorite_count', 'num_hashtags', 'num_mentions', 'num_urls',
              'user_id_conv']
df_tweets.loc[aggregated_one_text.index, attributes] = aggregated_one_text[attributes]

In [ ]:
# Drop duplicates which fit both criteria of duplication and misspelled user_id
indices_to_drop = duplicates_bool & ~df_tweets.index.isin(aggregated.index)
indices_to_drop.value_counts()

In [ ]:
df_tweets = df_tweets[~indices_to_drop]

In [ ]:
# Reset index
df_tweets = df_tweets.drop(columns=['index'])
df_tweets = df_tweets.reset_index()

In [ ]:
df_tweets.info(show_counts=True)

## File checkpoint

In [ ]:
df_tweets.to_csv(path_or_buf=DATA_PATH + 'tweets_no_dupl_1.3.csv', sep='#', index=False)

In [ ]:
df_tweets = pd.read_csv(DATA_PATH + 'tweets_no_dupl_1.3.csv', sep='#')

## Additional checks of duplicates

We check other combinations of attributes to find duplicates.

Most of the combinations don't have a significant number of values which could be considered duplicates, apart from the ['user_id', 'text'] combination. 

In [ ]:
for attr in ['id', ['id', 'user_id'], ['id', 'created_at'], ['id', 'user_id', 'created_at'], ['user_id', 'text'],
             ['id', 'user_id', 'text']]:
    counts = df_tweets.duplicated(attr).value_counts()
    if len(counts) > 1:
        dupl_count = counts[1]
    else:
        dupl_count = 0
    print(f"{attr} {dupl_count}")

# 1.4 [user_id,text]

In [ ]:
df_tweets.info()

The duplicates by ['user_id', 'text'] are more than one million, which is a considerable amount and is worth further investigation. 

It's surely possible that a user, especially if it is a bot, tweets many times the same text. What we want to check is just that among these duplicates all the dates are valid, which is, in a range that goes from the Twitter foundation up to september 2022.

If the dates are valid, we keep the tweets. Otherwise it is considered a noisy duplicate and is removed.

In [ ]:
twitter_foundation = pd.to_datetime(["20060321"]).astype(np.int64)[0]
sep_2022 = pd.to_datetime(["20220915"]).astype(np.int64)[0]

In [ ]:
def wrong_date(date):
    return (date < twitter_foundation) | (date > sep_2022)

In [ ]:
# Cast dates to seconds
df_tweets['created_at_conv'] = pd.to_datetime(df_tweets['created_at']).astype(np.int64)

In [ ]:
# Criterion for duplicates
duplicates_bool = df_tweets.duplicated(['user_id', 'text'], keep=False)
duplicates_bool.value_counts()

In [ ]:
# Replace incorrect values with NaN to check how many they are
for attr in ['retweet_count', 'reply_count', 'favorite_count', 'num_hashtags', 'num_mentions', 'num_urls']:
    df_tweets[attr].replace(-1, np.NaN)

# All records which are potential duplicates and have a wrong date 
df_tweets[duplicates_bool][wrong_date(df_tweets.created_at_conv)].info(show_counts=True)

We will not aggregate the counters using a max operator as in previous steps, since the multiple "copies" in this case could be just a periodic tweet. Furthermore, there is no null value in the counters.
Given that there is no null value in the other fields, we will simply remove the records having a wrong date.

In [ ]:
df_tweets[duplicates_bool].created_at_conv.apply(wrong_date).value_counts()

In these duplicates 97602 records have wrong created_at.

In [ ]:
indices_to_drop = ~(duplicates_bool & df_tweets[duplicates_bool].created_at_conv.apply(wrong_date))
indices_to_drop.value_counts()

In [ ]:
df_tweets = df_tweets[indices_to_drop]

## File checkpoint

In [ ]:
df_tweets = df_tweets.drop(columns='index').reset_index()
df_tweets.to_csv(path_or_buf=DATA_PATH + 'tweets_no_dupl_1.4.csv', sep='#', index=False)

In [ ]:
df_tweets = pd.read_csv(DATA_PATH + 'tweets_no_dupl_1.4.csv', sep='#')

# 1.5 [user_id, created_at]

Considering additional combinations of attributes, the most relevant number of duplicates regards the ['user_id', 'created_at'] combination of attributes (['user_id', 'text'] was already analyzed).

In [ ]:
for attr in ['id', ['id', 'user_id'], ['id', 'created_at'], ['user_id', 'created_at'], ['created_at', 'text'],
             ['id', 'user_id', 'created_at'], ['user_id', 'text'], ['id', 'user_id', 'text']]:
    counts = df_tweets.duplicated(attr).value_counts()
    if len(counts) > 1:
        dupl_count = counts[1]
    else:
        dupl_count = 0
    print(f"{attr} {dupl_count}")

In [ ]:
# Criterion to select duplicates, considering non-null values
duplicates_bool = df_tweets.duplicated(['user_id', 'created_at'], keep=False) & ~df_tweets.user_id.isnull()
duplicates_bool.value_counts()

The duplicates that have non-null text also have all non-null counters

In [ ]:
df_tweets[duplicates_bool & ~df_tweets.text.isnull()].info(show_counts=True)

In [ ]:
# Duplicates with null text
df_tweets[duplicates_bool & df_tweets.text.isnull()].info(show_counts=True)

The duplicates removal procedeed with the merge procedure as in 1.2

In [ ]:
# Replace null values with -1
for a in ['retweet_count', 'reply_count', 'favorite_count', 'num_hashtags', 'num_urls', 'num_mentions']:
    df_tweets[a] = df_tweets[a].replace(np.NaN, -1)

In [ ]:
'''
For duplicates it checks if the texts of the different records are:
- if only one is non-null it outputs this text
- all NaN it outputs NaN
- if all are non-null returns the list of all texts
'''
def select_non_null_text(texts):
    nonnull_texts = []
    for text in texts:
        if ~pd.Series(text).isnull()[0]:
            nonnull_texts.append(text)
    if len(nonnull_texts) == 1:
        return nonnull_texts[0]
    elif len(nonnull_texts) == 0:
        return np.NaN
    elif len(nonnull_texts) > 1:
        return nonnull_texts

In [ ]:
# Reset index
df_tweets = df_tweets.drop(columns='index').reset_index()

In [ ]:
groupby_userid_createdat = df_tweets[duplicates_bool].groupby(['user_id', 'created_at'])

In [ ]:
# Save indices of duplicated tweets in a list for each record
aggregated = groupby_userid_createdat.agg(
    retweet_count=('retweet_count', max),
    reply_count=('reply_count', max),
    favorite_count=('favorite_count', max),
    num_hashtags=('num_hashtags', max),
    num_urls=('num_urls', max),
    num_mentions=('num_mentions', max),
    text=('text', select_non_null_text),
    Keep_index=('index', min),
    all_indices=('index', list)).reset_index()

In [ ]:
aggregated

In [ ]:
# Indices to keep
aggregated.index = aggregated['Keep_index']

In [ ]:
'''
Records which have as text_type a string were the ones which had a single non-null text,
while the others contain the list of texts for the duplicate records
'''
aggregated['text_type'] = aggregated.text.apply(type)
aggregated.text_type.value_counts()

In [ ]:
# All records which had only one clean tweet, and the other ones were noisy
aggregated_one_text = aggregated[aggregated.text_type == str]

In [ ]:
aggregated[aggregated.text_type == list].all_indices

In [ ]:
# Tweets which have multiple non-null texts are to keep because we can't determine if they are noisy
not_replaceable = np.concatenate(aggregated[aggregated.text_type == list].all_indices.values)

In [ ]:
attributes = ['retweet_count', 'reply_count', 'favorite_count', 'num_hashtags', 'num_mentions', 'num_urls', 'text']
df_tweets.loc[aggregated_one_text.index, attributes] = aggregated_one_text[attributes]

In [ ]:
'''
Select all records which are the noisy counterparts of records with only one clean text
or
all records which are replaceable
'''
indices_to_keep = duplicates_bool.index.isin(aggregated_one_text.index) | duplicates_bool.index.isin(not_replaceable) | ~duplicates_bool
# duplicates_bool.index.isin(aggregated_one_text.index) | duplicates_bool.index.isin(not_replaceable))

In [ ]:
# Selects only the records to keep
df_tweets = df_tweets[indices_to_keep]

In [ ]:
df_tweets = df_tweets.reset_index()
df_tweets = df_tweets.drop(columns=['index'])

In [ ]:
df_tweets.info(show_counts=True)

## File checkpoint

In [ ]:
df_tweets.to_csv(path_or_buf=DATA_PATH + 'tweets_no_dupl_1.5.csv', sep='#', index=False)

In [ ]:
df_tweets = pd.read_csv(DATA_PATH + 'tweets_no_dupl_1.5.csv', sep='#')

# Last checks

In [ ]:
for attr in ['id', ['id', 'user_id'], ['id', 'created_at'], ['id', 'text'], ['id', 'user_id', 'text'], ['id','user_id','created_at'], \
            ['user_id', 'created_at'], ['user_id', 'text'], ['created_at', 'text']]:
    counts = df_tweets.duplicated(attr).value_counts()
    if len(counts) > 1:
        dupl_count = counts[1]
    else:
        dupl_count = 0
    print(f"{attr} {dupl_count}")

In [ ]:
df_tweets[df_tweets.duplicated(['id', 'user_id', 'text'], keep=False)].info()